# Descripción del proyecto 
En el contexto de la creciente popularidad de la lectura en línea tras la pandemia, han surgido nuevas aplicaciones diseñadas para optimizar este servicio. Por ello, el objetivo principal de este proyecto es analizar información de la base de datos de uno de los principales competidores en este mercado, con el fin de desarrollar una propuesta de valor para un nuevo producto. El análisis se centrará en datos relacionados con libros, editoriales, autores, calificaciones de clientes y reseñas, con el propósito de garantizar una propuesta de alta calidad que satisfaga las necesidades de los usuarios.

In [28]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Estudiar las tablas

In [29]:
#imprimir primeras filas de books

books_query = 'select * from books'
books = pd.io.sql.read_sql(books_query, con = engine)

books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [30]:
#imprimir primeras filas de authors

authors_query = 'select * from authors'
authors = pd.io.sql.read_sql(authors_query, con = engine)

authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [31]:
#imprimir primeras filas de publishers

publishers_query = 'select * from publishers'
publishers = pd.io.sql.read_sql(publishers_query, con = engine)

publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [32]:
#imprimir primeras filas de ratings

ratings_query = 'select * from ratings'
ratings = pd.io.sql.read_sql(ratings_query, con = engine)

ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [33]:
#imprimir primeras filas de reviews

reviews_query = 'select * from reviews'
reviews = pd.io.sql.read_sql(reviews_query, con = engine)

reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Extracción de información

In [34]:
#encontrar el número de libros publicados después del 1 de enero de 2000

with engine.connect() as connection:
    query = text("""
        SELECT 
            COUNT(publication_date)
        FROM 
            books 
        WHERE 
            publication_date > '2000-01-01'
    """) 

    result = connection.execute(query)

    for fila in result:
        print(f"La cantidad de libros publicados después del 1 de enero de 2000 es: {fila[0]}")

La cantidad de libros publicados después del 1 de enero de 2000 es: 819


In [35]:
# encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.

with engine.connect() as connection:
    query = text("""
        SELECT 
            books.book_id,
            books.title,
            COUNT(reviews.review_id) AS reviews_count,
            AVG(ratings.rating)  AS avg_rating                
        FROM 
            books
            INNER JOIN reviews on reviews.book_id = books.book_id
            INNER JOIN ratings on ratings.book_id = books.book_id        
        GROUP BY 
            books.book_id, books.title
        ORDER BY 
            avg_rating desc
        """) 

    result = connection.execute(query)
    
    for fila in result:
        id, title, reviews_count, avg_rating = fila
        print(f"{title} (ID: {id}) tiene {reviews_count} reseñas y un rating promedio de {float(avg_rating):.2f}.")

A Woman of Substance (Emma Harte Saga #1) (ID: 55) tiene 4 reseñas y un rating promedio de 5.00.
Pop Goes the Weasel (Alex Cross  #5) (ID: 518) tiene 4 reseñas y un rating promedio de 5.00.
In the Hand of the Goddess (Song of the Lioness  #2) (ID: 347) tiene 6 reseñas y un rating promedio de 5.00.
Piercing the Darkness (Darkness  #2) (ID: 513) tiene 4 reseñas y un rating promedio de 5.00.
Act of Treason (Mitch Rapp  #9) (ID: 57) tiene 4 reseñas y un rating promedio de 5.00.
The Big Bad Wolf  (Alex Cross  #9) (ID: 642) tiene 4 reseñas y un rating promedio de 5.00.
My Name Is Asher Lev (ID: 459) tiene 4 reseñas y un rating promedio de 5.00.
Wherever You Go  There You Are: Mindfulness Meditation in Everyday Life (ID: 972) tiene 4 reseñas y un rating promedio de 5.00.
Dead Souls (ID: 182) tiene 4 reseñas y un rating promedio de 5.00.
March (ID: 418) tiene 4 reseñas y un rating promedio de 5.00.
Welcome to Temptation (Dempseys  #1) (ID: 967) tiene 4 reseñas y un rating promedio de 5.00.
Wom

In [36]:
# identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.

with engine.connect() as connection:
    query = text("""
        SELECT 
            books.publisher_id,
            publishers.publisher,
            COUNT(book_id) AS published_books                          
        FROM 
            books 
            LEFT JOIN publishers on publishers.publisher_id = books.publisher_id
        WHERE
            num_pages > 50      
        GROUP BY 
            books.publisher_id, publishers.publisher
        ORDER BY 
            published_books desc
        LIMIT 
            1
        """) 
    
    result = connection.execute(query)

    for fila in result:
        id, publisher, num_books = fila 
        print(f"{publisher} (ID: {id}) ha sido la editorial que más libros ha publicado, {num_books} en total.")

Penguin Books (ID: 212) ha sido la editorial que más libros ha publicado, 42 en total.


In [37]:
#identificar al autor que tiene la más alta calificación promedio del libro (calificaciones >= 50)

with engine.connect() as connection:
    query = text("""
        SELECT 
            books.author_id,
            authors.author,
            books.title,
            COUNT(ratings.rating_id) as rating_count,
            AVG(ratings.rating) AS avg_rating               
        FROM 
            books
            LEFT JOIN authors on authors.author_id = books.author_id
            LEFT JOIN ratings on ratings.book_id = books.book_id 
        GROUP BY 
            books.author_id, authors.author, books.title
        HAVING 
            COUNT(ratings.rating_id) >= 50
        ORDER BY 
            avg_rating desc
        LIMIT 
            1
        """) 

    result = connection.execute(query)

    for fila in result:
        author_id, author, title,  rating_count, avg_rating = fila
        print(f"{author} (ID: {author_id}) es quien tiene el rating más alto de {float(avg_rating):.2f} con el libro {title}.")

J.K. Rowling/Mary GrandPré (ID: 236) es quien tiene el rating más alto de 4.41 con el libro Harry Potter and the Prisoner of Azkaban (Harry Potter  #3).


In [38]:
# encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

with engine.connect() as connection:
    query = text("""
        SELECT 
            AVG(subquery_1.reviews_count) AS avg_reviews     
        FROM 
            (SELECT
                ratings.username AS username,
                COUNT(books.book_id) AS books_count,
                COUNT(reviews.review_id) AS reviews_count
            FROM
                books
                INNER JOIN ratings ON books.book_id = ratings.book_id
                INNER JOIN reviews ON ratings.username = reviews.username
            GROUP BY
                ratings.username
            HAVING 
                COUNT(DISTINCT books.book_id) > 50
            ) AS subquery_1;
        """) 

    result = connection.execute(query)

    for fila in result:
        print(f"El número promedio de revisiones por usuario que calificaron más de 50 libros es de {int(fila[0])}")


El número promedio de revisiones por usuario que calificaron más de 50 libros es de 1326


## Conclusiones 

1. **Cantidad de libros publicados recientemente**  
   Desde el 1 de enero del año 2000, se han publicado 819 libros en la base de datos analizada. Esto muestra un mercado activo con alta oferta  

2. **Libros con calificaciones perfectas**  
   Se identificaron libros con un promedio de calificación de 5.00 y otros con gran cantidad de reseñas lo que indica una alta satisfacción de los lectores.  

3. **Editorial más prolífica**  
   Penguin Books lidera en cantidad de publicaciones, con 42 libros en total. Esta editorial podría ser considerada para el desarrollo del nuevo producto.  

4. **Autores destacados**  
   J.K. Rowling/Mary GrandPré obtienen el promedio de calificación más alto (4.41) con el libro *Harry Potter and the Prisoner of Azkaban (Harry Potter #3)*. Esto evidencia el impacto de autores reconocidos en el mercado.  

5. **Usuarios altamente comprometidos**  
   Los usuarios que han calificado más de 50 libros tienen, en promedio, 1,326 revisiones. Estos usuarios son clave debido a su alto nivel de interacción con la plataforma.  

6. **Oportunidades para el nuevo producto**  
   El análisis muestra que los títulos mejor valorados, las editoriales con más publicaciones y los usuarios más activos son puntos clave. Esto permite plantear un producto que resalte libros populares, autores reconocidos y ofrezca una experiencia acorde a las preferencias de los lectores.  